A notebook where we synthesize regression models across conditions.  We examine the performance of the models when they are fit together and individually. 

Specifically, we synthesize regression models that predict only one variable.  The models project data down to a single variable, which is then passed through a complicated non-linear function.  When we say we synthesize models "across conditions" we mean that we simulate data so that data for each model projects into a given range of the low-s space, so we only see the behavior of the non-linear function in a certain range for each individual. 

Some details:

1) Each input variable in the model is associated with a 2-d position (position is the measurable property in this example).  Variable positions are sampled uniformly from the unit square. 

2) We pull the weights for each model from a prior where the mean and std. conditioned on position are sums of bump functions (so they are truly smooth). When fitting models, we use hypercube functions for the mean and std. (so there
is model mismatch here).  This can make correctly learning the std functions tricky, because if the true mean functions vary alot within a single hypercube, we will learn a standard deviation that is elevated in that region

3) We simulate data with scales and biases which are very close to 1 and 0, respectively.  This keeps things simpler.  When fitting, we use priors which assume scales and biases are concentrated near these values. 

4) We pull noise variances from a Gamma distribution when simulating data, and we also fit models with priors and posteriors which are also Gamma distributions over noise variances.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import math

import matplotlib
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import numpy.linalg
import torch

from janelia_core.math.basic_functions import list_grid_pts
from janelia_core.ml.extra_torch_modules import DenseLNLNet
from janelia_core.ml.extra_torch_modules import QuadSurf
from janelia_core.ml.utils import list_torch_devices


from probabilistic_model_synthesis.gaussian_nonlinear_regression import compare_weight_prior_dists
from probabilistic_model_synthesis.gaussian_nonlinear_regression import align_low_d_spaces
from probabilistic_model_synthesis.gaussian_nonlinear_regression import fit_with_hypercube_priors
from probabilistic_model_synthesis.gaussian_nonlinear_regression import Fitter
from probabilistic_model_synthesis.gaussian_nonlinear_regression import generate_hypercube_prior_collection
from probabilistic_model_synthesis.gaussian_nonlinear_regression import GNLRMdl
from probabilistic_model_synthesis.simulation import generate_sum_of_bump_fcns_dist
from probabilistic_model_synthesis.simulation import cone_and_projected_interval_sample
from probabilistic_model_synthesis.simulation import sample_proj_data_from_interval
from probabilistic_model_synthesis.simulation import sample_proj_data_from_interval
from probabilistic_model_synthesis.visualization import assign_colors_to_pts
from probabilistic_model_synthesis.visualization import plot_three_dim_pts

In [3]:
%matplotlib notebook
plt.style.use('dark_background')
matplotlib.rcParams.update({'font.size': 20})

## Parameters go here

In [4]:
# Range of the number of input variables we observe from each individual - the actual number of variables we 
# observe from an individual will be pulled uniformly from this range (inclusive)
n_input_var_range = [5000, 5100]

# Range of the number of samples we observe from each individual - the actual number we observe from each individual
# will be unformly from this range (inclusive)
n_smps_range = [12000, 15000]

# ===============================================================================================
# Parameters for the true priors 

# Options for the prior distribution on weights
true_w_prior_opts = {'n_bump_fcns': 50, 'd_in': 2, 'p': 1, 'mn_m_std': 1.0, 'std_m_std': .1, 'bump_w': .2}

# Options for the prior distributions on the scales, biases and psi
true_s_in_prior_opts = {'mn_mn': 1.0/np.sqrt(n_input_var_range[0]), 'mn_std': .00000001, 
                        'std_iv': .000001, 'std_lb': .00000001}

true_s_out_prior_opts = {'mn_mn': 1.0, 'mn_std': .00000001, 
                         'std_iv': .000001, 'std_lb': .00000001}

true_b_prior_opts = {'mn_mn': 0.0, 'mn_std': .00000001, 'std_iv': .000001, 'std_lb': .00000001}
true_psi_prior_opts = {'conc_iv': 10.0, 'rate_iv': 1000.0, 'rate_ub': 10000.0}

# ===============================================================================================
# Parameters for the fit models

# The full options for setting up the prior on weights
fit_hc_params = {'n_divisions_per_dim': [20, 20], 
                 'dim_ranges': np.asarray([[-.1, 1.1],
                                       [-.1, 1.1]]),
                 'n_div_per_hc_side_per_dim': [1, 1]}

# The full options for setting up the prior on weights
fit_w_prior_opts = {'mn_hc_params': fit_hc_params, 'std_hc_params': fit_hc_params, 
                     'min_std': .000001, 'mn_init': 0.0, 'std_init': .3}

# Options for the prior distributions on the scales, biases and psi
fit_s_in_prior_opts = true_s_in_prior_opts
fit_s_out_prior_opts = true_s_out_prior_opts
fit_b_prior_opts = true_b_prior_opts
fit_psi_prior_opts = true_psi_prior_opts

# Options for posterior distribtions 
s_in_post_opts = {'mn_mn': 1.0/np.sqrt(n_input_var_range[0]), 'mn_std': .000001, 
               'std_iv': .00001, 'std_lb': .0000001}
s_out_post_opts = {'mn_mn': 1.0, 'mn_std': .000001, 
               'std_iv': .00001, 'std_lb': .0000001}

b_post_opts = {'mn_mn': 0.0, 'mn_std': .01,'std_iv': .00001, 'std_lb': .0000001}
psi_post_opts = {'conc_iv': 10.0, 'rate_iv': 1.0, 'rate_ub': 100000.0}

# Options for the densenet which makes up the shared-m module
dense_net_opts = {'n_layers': 10, 'growth_rate': 5, 'bias': True}

# ======================================================================================================
# Parameters for fitting - should be entered as lists, each entry corresponding to one round of fitting

# Parameters when fitting combined models
comb_sp_fit_opts = [{'n_epochs': 100, 'milestones': None, 'update_int': 100, 'init_lr': .1}]
comb_ip_fit_opts = [{'n_epochs': 5000, 'milestones': [500], 'update_int': 100, 'init_lr': .1}]

single_sp_fit_opts = [{'n_epochs': 100, 'milestones': None, 'update_int': 100, 'init_lr': .10}]
single_ip_fit_opts = [{'n_epochs': 5000, 'milestones': [500], 'update_int': 100, 'init_lr': .10}, 
                      {'n_epochs': 5000, 'milestones': None, 'update_int': 100, 'init_lr': .01}] 


# ======================================================================================================
# Specify the number of intermediate variables and number of variables we predict
p = 1
d_pred = 1


In [5]:
# Range of the number of input variables we observe from each individual - the actual number of variables we 
# observe from an individual will be pulled uniformly from this range (inclusive)
n_input_var_range = [5000, 5100]

# Range of the number of samples we observe from each individual - the actual number we observe from each individual
# will be unformly from this range (inclusive)
n_smps_range = [12000, 15000]

# ===============================================================================================
# Parameters for the true priors 

# Options for the prior distribution on weights
true_w_prior_opts = {'n_bump_fcns': 50, 'd_in': 2, 'p': 1, 'mn_m_std': 1.0, 'std_m_std': .1, 'bump_w': .2}

# Options for the prior distributions on the scales, biases and psi
true_s_in_prior_opts = {'mn_mn': 1.0/np.sqrt(n_input_var_range[0]), 'mn_std': .00000001, 
                     'std_iv': .00001, 'std_lb': .0000001}

true_s_out_prior_opts = {'mn_mn': 1.0, 'mn_std': .00000001, 
                     'std_iv': .00001, 'std_lb': .0000001}

true_b_prior_opts = {'mn_mn': 0.0, 'mn_std': .00000001, 'std_iv': .001}
true_psi_prior_opts = {'conc_iv': 10.0, 'rate_iv': 1000.0, 'rate_ub': 10000.0}

# ===============================================================================================
# Parameters for the fit models

# The full options for setting up the prior on weights
fit_hc_params = {'n_divisions_per_dim': [20, 20], 
                 'dim_ranges': np.asarray([[-.1, 1.1],
                                       [-.1, 1.1]]),
                 'n_div_per_hc_side_per_dim': [1, 1]}

# The full options for setting up the prior on weights
fit_w_prior_opts = {'mn_hc_params': fit_hc_params, 'std_hc_params': fit_hc_params, 
                     'min_std': .000001, 'mn_init': 0.0, 'std_init': .3}

# Options for the prior distributions on the scales, biases and psi
fit_s_in_prior_opts = true_s_in_prior_opts
fit_s_out_prior_opts = true_s_out_prior_opts
fit_b_prior_opts = true_b_prior_opts
fit_psi_prior_opts = true_psi_prior_opts

# Options for posterior distribtions 
s_in_post_opts = {'mn_mn': 1.0/np.sqrt(n_input_var_range[0]), 'mn_std': .000001, 
               'std_iv': .00001, 'std_lb': .0000001}
s_out_post_opts = {'mn_mn': 1.0, 'mn_std': .000001, 
               'std_iv': .00001, 'std_lb': .0000001}

b_post_opts = {'mn_mn': 0.0, 'mn_std': .01}
psi_post_opts = {'conc_iv': 10.0, 'rate_iv': 1.0, 'rate_ub': 100000.0}

# Options for the densenet which makes up the shared-m module
dense_net_opts = {'n_layers': 10, 'growth_rate': 5, 'bias': True}

# ======================================================================================================
# Parameters for fitting - should be entered as lists, each entry corresponding to one round of fitting

# Parameters when fitting combined models
comb_sp_fit_opts = [{'n_epochs': 10, 'milestones': [500], 'update_int': 100, 'init_lr': .1}]
comb_ip_fit_opts = [{'n_epochs': 5000, 'milestones': [500], 'update_int': 100, 'init_lr': .1},
                    {'n_epochs': 5000, 'milestones': None, 'update_int': 100, 'init_lr': .01}] 

single_sp_fit_opts = comb_sp_fit_opts
single_ip_fit_opts = comb_ip_fit_opts


# ======================================================================================================
# Specify the number of intermediate variables and number of variables we predict
p = 1
d_pred = 1


## Create true distributions that govern how systems under study are generated

In [6]:
# Because we do not use the hypercube priors on weights, we provide some default paramaters for these before
# creating the priors

temp_hc_params = {'n_divisions_per_dim': [10, 10], 
                  'dim_ranges': np.asarray([[-.1, 1.1],
                                       [-.1, 1.1]]),
                   'n_div_per_hc_side_per_dim': [1, 1]}

temp_w_prior_opts = {'mn_hc_params': temp_hc_params, 'std_hc_params': temp_hc_params, 
                     'min_std': .000001, 'mn_init': 0.0, 'std_init': .3}

true_priors = generate_hypercube_prior_collection(p=p, d_pred=d_pred, 
                                                  w_prior_opts=temp_w_prior_opts, 
                                                  s_in_prior_opts=true_s_in_prior_opts, 
                                                  b_in_prior_opts=true_b_prior_opts, 
                                                  s_out_prior_opts=true_s_out_prior_opts,
                                                  b_out_prior_opts=true_b_prior_opts, 
                                                  psi_prior_opts=true_psi_prior_opts)

# We replace the prior over weights 
true_priors.w_prior = generate_sum_of_bump_fcns_dist(**true_w_prior_opts)

## Define the true non-linear function relating projections of input variables to the mean of output variables

In [7]:
class Quad(torch.nn.Module):
    
    def forward(self, x):
        return x + torch.sin(3*x) 

In [8]:
m_true = Quad()

## Generate data

### Generate properties

In [9]:
n_individuals = 4
ind_n_vars = np.random.randint(n_input_var_range[0], n_input_var_range[1]+1, n_individuals)
ind_props = [torch.rand(size=[n_vars,2]) for n_vars in ind_n_vars]

### Generate true models for each individual

In [10]:
with torch.no_grad():
    ind_true_mdls = [GNLRMdl(m=m_true, 
                     w=true_priors.w_prior.form_standard_sample(true_priors.w_prior.sample(props)),
                     s_in=true_priors.s_in_prior.form_standard_sample(true_priors.s_in_prior.sample(props)).squeeze(axis=1),
                     b_in=true_priors.b_in_prior.form_standard_sample(true_priors.b_in_prior.sample(props)).squeeze(axis=1),
                     s_out=true_priors.s_out_prior.form_standard_sample(true_priors.s_out_prior.sample(props)).squeeze(axis=1),
                     b_out=true_priors.b_out_prior.form_standard_sample(true_priors.b_out_prior.sample(props)).squeeze(axis=1),
                     psi=true_priors.psi_prior.form_standard_sample(true_priors.psi_prior.sample(props)).squeeze(axis=1))
                     for props in ind_props]
    
    if d_pred > 1:
        for mdl in ind_true_mdls:
            mdl.s_out.data = mdl.s_out.data.squeeze()
            mdl.b_out.data = mdl.b_out.data.squeeze()
            mdl.psi.data = mdl.psi.data.squeeze()
                             

### Generate observations from true models

In [11]:
ang_ranges = np.linspace(0, 2*np.pi, n_individuals+1)

In [12]:
ang_ranges

array([0.        , 1.57079633, 3.14159265, 4.71238898, 6.28318531])

In [13]:
ind_n_smps = np.random.randint(n_smps_range[0], n_smps_range[1]+1, n_individuals)
ind_data = [None]*n_individuals
ind_data_new = [None]*n_individuals

intervals = 1.96*np.sqrt(n_input_var_range[0])*np.linspace(-1, 1, n_individuals+1)
ang_ranges = np.linspace(0, 2*np.pi, n_individuals+1)
for i in range(n_individuals):
    
    cur_range = copy.deepcopy(ang_ranges[i:i+2])
    #cur_range[-1] += np.pi
    print(cur_range)

    
    x_i_new = cone_and_projected_interval_sample(n_smps=ind_n_smps[i], 
                                             locs=ind_props[i].numpy(),
                                             ctr=np.asarray([.5, .5]), 
                                             ang_range=cur_range,
                                             w=ind_true_mdls[i].w.detach().numpy(),
                                             interval=[intervals[i], intervals[i+1]], 
                                             big_std=np.sqrt(n_individuals), 
                                             small_std=.001)#np.sqrt(n_individuals)) #.0001)
    x_i_new = torch.tensor(x_i_new, dtype=torch.float)
    
    with torch.no_grad():
        y_i_new = ind_true_mdls[i].sample(x=x_i_new)
    ind_data_new[i] = (x_i_new, y_i_new)
    
    
    x_i = torch.tensor(sample_proj_data_from_interval(n_smps=ind_n_smps[i], 
                                                      w=ind_true_mdls[i].w.detach().numpy(), 
                                                      interval=[intervals[i], intervals[i+1]]),
                       dtype=torch.float)
    
    
    with torch.no_grad():
        y_i = ind_true_mdls[i].sample(x=x_i)
    ind_data[i] = (x_i, y_i)
    
    print('Done generating data for individual ' + str(i) + '.')
    

[0.         1.57079633]
n_big_vars: 1290
n_small_vars: 3805
Changed 7
Done generating data for individual 0.
[1.57079633 3.14159265]
n_big_vars: 1238
n_small_vars: 3806
Changed 7
Done generating data for individual 1.
[3.14159265 4.71238898]
n_big_vars: 1210
n_small_vars: 3803
Changed 7
Done generating data for individual 2.
[4.71238898 6.28318531]
n_big_vars: 1233
n_small_vars: 3834
Changed 7
Done generating data for individual 3.


In [14]:
torch.std(x_i, dim=0)

tensor([0.9983, 1.0006, 0.9868,  ..., 1.0061, 1.0060, 1.0123])

## Synthesize models (fit models to the data together)

In [15]:
comb_fit_rs = fit_with_hypercube_priors(data=ind_data, props=ind_props, p=p, 
                                   w_prior_opts=fit_w_prior_opts, 
                                   s_in_prior_opts=fit_s_in_prior_opts,
                                   b_in_prior_opts=fit_b_prior_opts,
                                   s_out_prior_opts=fit_s_out_prior_opts,
                                   b_out_prior_opts=fit_b_prior_opts,
                                   psi_prior_opts=fit_psi_prior_opts,
                                   s_in_post_opts=s_in_post_opts,
                                   b_in_post_opts=b_post_opts,
                                   s_out_post_opts=s_out_post_opts,
                                   b_out_post_opts=b_post_opts,
                                   psi_post_opts=psi_post_opts,
                                   dense_net_opts=dense_net_opts, 
                                   sp_fit_opts=comb_sp_fit_opts, 
                                   ip_fit_opts=comb_ip_fit_opts)
                        

Found 1 GPUs
Beginning SP fitting.

=========== EPOCH 0 COMPLETE ===========
Obj: 2.03e+08
----------------------------------------
NELL: 3.55e+04, 3.38e+04, 4.06e+04, 3.58e+04
W KL: 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00
S_in KL: 2.53e+07, 2.53e+07, 2.53e+07, 2.53e+07
B_in KL: 2.42e+03, 2.22e+03, 2.21e+03, 2.35e+03
S_out KL: 2.53e+07, 2.53e+07, 2.53e+07, 2.53e+07
B_out KL: 1.93e+03, 1.77e+03, 1.92e+03, 2.13e+03
Psi KL: 7.78e+03, 7.78e+03, 7.78e+03, 7.78e+03
----------------------------------------
LR: 0.1
Elapsed time (secs): 0.29241204261779785
----------------------------------------
CPU cur memory used (GB): 4.60e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00
Beginning IP fitting.

=========== EPOCH 0 COMPLETE ===========
Obj: 1.34e+08
----------------------------------------
NELL: 8.48e+05, 3.30e+04, 2.47e+04, 5.42e+04
W KL: 5.91e+02, 5.85e+02, 5.78e+02, 5.84e+02
S_in KL: 1.66e+07, 1.66e+07, 1.66e+07, 1.66e+07
B_in KL: 1.31e+03, 2.28e+03, 1.45e+03, 1.88e+


=========== EPOCH 1200 COMPLETE ===========
Obj: 2.03e+04
----------------------------------------
NELL: 3.16e+03, 1.01e+04, -1.26e+04, -9.58e+03
W KL: 3.99e+03, 3.52e+03, 1.23e+04, 9.09e+03
S_in KL: 8.74e+00, 2.93e+00, 1.09e+00, 1.38e+01
B_in KL: 2.64e-02, 4.18e+01, 2.67e+01, 1.13e-01
S_out KL: 2.15e-02, 9.68e-03, 6.66e-03, 1.10e-03
B_out KL: 7.35e+01, 1.77e+02, 1.58e+01, 6.11e-01
Psi KL: 4.95e-01, 2.98e+00, 3.27e+00, 2.78e+00
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 107.51117157936096
----------------------------------------
CPU cur memory used (GB): 5.62e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00

=========== EPOCH 1300 COMPLETE ===========
Obj: 1.91e+04
----------------------------------------
NELL: 2.40e+03, 1.09e+04, -1.27e+04, -9.40e+03
W KL: 3.36e+03, 3.19e+03, 1.21e+04, 8.78e+03
S_in KL: 4.69e-01, 1.20e+00, 9.00e+00, 4.92e+01
B_in KL: 3.07e-01, 2.07e+01, 8.61e+00, 1.72e-01
S_out KL: 2.47e-03, 1.04e-02


=========== EPOCH 2400 COMPLETE ===========
Obj: -1.53e+03
----------------------------------------
NELL: -5.18e+03, -1.04e+04, -1.26e+04, -7.81e+03
W KL: 4.86e+03, 1.08e+04, 1.12e+04, 6.89e+03
S_in KL: 8.85e+01, 1.69e+02, 2.84e+01, 1.24e+02
B_in KL: 6.76e+00, 5.86e+01, 5.69e+01, 5.05e+00
S_out KL: 2.11e-03, 1.40e-02, 1.43e-01, 9.97e-02
B_out KL: 9.58e+00, 9.68e+01, 7.62e+01, 8.55e+00
Psi KL: 9.71e-01, 1.88e+00, 2.37e+00, 1.82e+00
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 214.80403089523315
----------------------------------------
CPU cur memory used (GB): 5.62e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00

=========== EPOCH 2500 COMPLETE ===========
Obj: 7.90e+03
----------------------------------------
NELL: 4.51e+02, -1.00e+04, -1.25e+04, -8.07e+03
W KL: 5.57e+03, 1.05e+04, 1.11e+04, 6.99e+03
S_in KL: 3.48e+03, 2.53e+01, 1.97e+01, 2.46e+01
B_in KL: 3.07e+01, 8.40e+00, 8.72e+00, 5.11e+00
S_out KL: 1.32e-01, 3.20


=========== EPOCH 3600 COMPLETE ===========
Obj: -2.36e+03
----------------------------------------
NELL: -7.07e+03, -1.07e+04, -1.23e+04, -7.47e+03
W KL: 5.70e+03, 1.08e+04, 1.11e+04, 6.90e+03
S_in KL: 2.82e+01, 4.35e+01, 1.90e+01, 1.86e+02
B_in KL: 5.58e+00, 3.92e+01, 3.44e+01, 2.77e+00
S_out KL: 7.79e+00, 1.22e+01, 6.68e+01, 8.52e-01
B_out KL: 2.05e+01, 1.11e+02, 8.78e+01, 9.24e+00
Psi KL: 5.95e-01, 1.68e+00, 2.32e+00, 1.52e+00
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 322.1742465496063
----------------------------------------
CPU cur memory used (GB): 5.62e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00

=========== EPOCH 3700 COMPLETE ===========
Obj: 1.45e+04
----------------------------------------
NELL: 1.22e+03, -1.07e+04, -1.23e+04, -7.19e+03
W KL: 6.19e+03, 1.08e+04, 1.11e+04, 6.72e+03
S_in KL: 6.96e+03, 2.03e+02, 1.20e+02, 8.06e+01
B_in KL: 8.68e+01, 9.92e+01, 7.39e+01, 4.06e+00
S_out KL: 5.89e+01, 3.52e


=========== EPOCH 4800 COMPLETE ===========
Obj: 5.36e+00
----------------------------------------
NELL: -6.93e+03, -1.07e+04, -1.21e+04, -7.15e+03
W KL: 6.04e+03, 1.08e+04, 1.11e+04, 7.05e+03
S_in KL: 3.18e+01, 1.95e+02, 1.95e+02, 5.43e+02
B_in KL: 3.43e+00, 4.28e+01, 3.98e+01, 1.59e+00
S_out KL: 2.24e+02, 2.77e-02, 2.45e+02, 1.13e+02
B_out KL: 2.94e+01, 1.01e+02, 7.60e+01, 1.48e+01
Psi KL: 1.19e+00, 1.44e+00, 2.49e+00, 1.25e+00
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 429.57702803611755
----------------------------------------
CPU cur memory used (GB): 5.62e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00

=========== EPOCH 4900 COMPLETE ===========
Obj: 3.35e+02
----------------------------------------
NELL: -6.97e+03, -1.06e+04, -1.18e+04, -7.84e+03
W KL: 6.04e+03, 1.08e+04, 1.11e+04, 6.98e+03
S_in KL: 1.92e+02, 2.47e+02, 4.08e+02, 6.57e+01
B_in KL: 3.49e+00, 5.23e+01, 5.97e+01, 7.28e+00
S_out KL: 6.70e-03, 3.44


=========== EPOCH 1100 COMPLETE ===========
Obj: 2.15e+03
----------------------------------------
NELL: -7.89e+03, -1.08e+04, -1.20e+04, -6.57e+03
W KL: 6.36e+03, 1.09e+04, 1.11e+04, 7.16e+03
S_in KL: 2.55e+01, 8.64e+01, 2.73e+02, 9.37e+02
B_in KL: 3.39e+00, 5.73e+01, 5.09e+01, 4.22e+00
S_out KL: 6.68e+01, 5.78e-01, 2.05e-02, 2.29e+03
B_out KL: 1.77e+01, 9.06e+01, 8.42e+01, 1.32e+01
Psi KL: 1.01e+00, 1.32e+00, 2.27e+00, 1.20e+00
----------------------------------------
LR: 0.01
Elapsed time (secs): 99.12233352661133
----------------------------------------
CPU cur memory used (GB): 5.62e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00

=========== EPOCH 1200 COMPLETE ===========
Obj: -7.47e+02
----------------------------------------
NELL: -7.96e+03, -1.07e+04, -1.17e+04, -8.06e+03
W KL: 6.62e+03, 1.08e+04, 1.11e+04, 7.21e+03
S_in KL: 1.14e+02, 1.97e+02, 6.85e+02, 5.50e+01
B_in KL: 5.19e+00, 6.48e+01, 5.54e+01, 5.48e+00
S_out KL: 4.86e+00, 3.53e+02, 8.09e+01, 


=========== EPOCH 2400 COMPLETE ===========
Obj: -1.02e+03
----------------------------------------
NELL: -8.77e+03, -1.08e+04, -1.21e+04, -7.04e+03
W KL: 7.03e+03, 1.08e+04, 1.11e+04, 6.89e+03
S_in KL: 7.07e+00, 3.39e+02, 8.91e+00, 4.72e+02
B_in KL: 3.25e+00, 5.25e+01, 5.48e+01, 5.21e+00
S_out KL: 9.28e+01, 4.10e+02, 1.40e+02, 1.04e+01
B_out KL: 3.67e+01, 1.17e+02, 9.30e+01, 1.41e+01
Psi KL: 7.56e-01, 1.48e+00, 2.55e+00, 1.86e+00
----------------------------------------
LR: 0.01
Elapsed time (secs): 216.90324878692627
----------------------------------------
CPU cur memory used (GB): 5.62e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00

=========== EPOCH 2500 COMPLETE ===========
Obj: 9.46e+03
----------------------------------------
NELL: -7.37e+03, -1.03e+04, -1.17e+04, -6.48e+03
W KL: 7.01e+03, 1.08e+04, 1.11e+04, 6.86e+03
S_in KL: 1.03e+03, 9.03e+02, 8.16e+02, 8.26e+02
B_in KL: 7.05e+00, 6.59e+01, 5.59e+01, 3.62e+00
S_out KL: 2.96e+02, 5.05e+03, 1.45e+02,


=========== EPOCH 3700 COMPLETE ===========
Obj: -5.82e+02
----------------------------------------
NELL: -8.92e+03, -1.05e+04, -1.20e+04, -8.10e+03
W KL: 7.16e+03, 1.08e+04, 1.11e+04, 6.94e+03
S_in KL: 1.06e+02, 9.50e+02, 2.40e+02, 4.57e+01
B_in KL: 4.65e+00, 6.61e+01, 6.28e+01, 7.36e+00
S_out KL: 7.36e+02, 1.44e+01, 2.76e-01, 4.68e+02
B_out KL: 8.92e+00, 1.05e+02, 7.90e+01, 1.20e+01
Psi KL: 9.61e-01, 1.31e+00, 2.47e+00, 1.38e+00
----------------------------------------
LR: 0.01
Elapsed time (secs): 335.0998773574829
----------------------------------------
CPU cur memory used (GB): 5.62e+00
GPU_0 cur memory used (GB): 1.02e+00, max memory used (GB): 1.02e+00

=========== EPOCH 3800 COMPLETE ===========
Obj: 1.79e+03
----------------------------------------
NELL: -8.53e+03, -1.06e+04, -1.23e+04, -7.46e+03
W KL: 7.19e+03, 1.08e+04, 1.11e+04, 6.98e+03
S_in KL: 2.97e+02, 1.12e+02, 3.37e+01, 2.20e+02
B_in KL: 2.84e+00, 4.69e+01, 5.13e+01, 4.69e+00
S_out KL: 1.05e+01, 3.42e+03, 4.69e+00, 

## Fit a model to one example individual

In [16]:
single_fit_ind = 0

In [17]:
single_fit_rs = fit_with_hypercube_priors(data=[ind_data_new[single_fit_ind]], 
                                          props=[ind_props[single_fit_ind]], p=p, 
                                          w_prior_opts=fit_w_prior_opts, 
                                          s_in_prior_opts=fit_s_in_prior_opts,
                                          b_in_prior_opts=fit_b_prior_opts,
                                          s_out_prior_opts=fit_s_out_prior_opts,
                                          b_out_prior_opts=fit_b_prior_opts,
                                          psi_prior_opts=fit_psi_prior_opts,
                                          s_in_post_opts=s_in_post_opts,
                                          b_in_post_opts=b_post_opts,
                                          s_out_post_opts=s_out_post_opts,
                                          b_out_post_opts=b_post_opts,
                                          psi_post_opts=psi_post_opts,
                                          dense_net_opts=dense_net_opts, 
                                          sp_fit_opts=single_sp_fit_opts, 
                                          ip_fit_opts=single_ip_fit_opts)

Found 1 GPUs
Beginning SP fitting.

=========== EPOCH 0 COMPLETE ===========
Obj: 5.07e+07
----------------------------------------
NELL: 3.32e+04
W KL: 0.00e+00
S_in KL: 2.53e+07
B_in KL: 2.34e+03
S_out KL: 2.53e+07
B_out KL: 2.34e+03
Psi KL: 7.78e+03
----------------------------------------
LR: 0.1
Elapsed time (secs): 0.023901939392089844
----------------------------------------
CPU cur memory used (GB): 6.64e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01
Beginning IP fitting.

=========== EPOCH 0 COMPLETE ===========
Obj: 3.33e+07
----------------------------------------
NELL: 5.39e+04
W KL: 4.82e+02
S_in KL: 1.66e+07
B_in KL: 1.48e+03
S_out KL: 1.66e+07
B_out KL: 2.39e+03
Psi KL: 4.70e+01
----------------------------------------
LR: 0.1
Elapsed time (secs): 0.037363290786743164
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 100 COMPLETE ===


=========== EPOCH 1700 COMPLETE ===========
Obj: 1.88e+02
----------------------------------------
NELL: -1.13e+01
W KL: 1.98e+02
S_in KL: 2.43e-02
B_in KL: 1.01e-02
S_out KL: 7.20e-05
B_out KL: 5.88e-02
Psi KL: 1.14e+00
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 42.75071620941162
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 1800 COMPLETE ===========
Obj: 1.46e+02
----------------------------------------
NELL: -5.52e+01
W KL: 2.00e+02
S_in KL: 2.21e-02
B_in KL: 1.38e-02
S_out KL: 1.80e-04
B_out KL: 3.24e-01
Psi KL: 8.71e-01
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 45.2611346244812
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 1900 COMPLETE ===========
Obj: 1.46e+0


=========== EPOCH 3500 COMPLETE ===========
Obj: -6.84e+03
----------------------------------------
NELL: -9.43e+03
W KL: 2.38e+03
S_in KL: 3.60e+00
B_in KL: 3.67e-01
S_out KL: 2.00e+02
B_out KL: 4.23e-01
Psi KL: 8.79e-01
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 88.00097823143005
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 3600 COMPLETE ===========
Obj: -6.81e+03
----------------------------------------
NELL: -9.55e+03
W KL: 2.45e+03
S_in KL: 2.58e+02
B_in KL: 1.48e+00
S_out KL: 3.41e+01
B_out KL: 1.08e+00
Psi KL: 4.99e-01
----------------------------------------
LR: 0.010000000000000002
Elapsed time (secs): 90.52867674827576
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 3700 COMPLETE ===========
Obj: -7.5


=========== EPOCH 300 COMPLETE ===========
Obj: -8.60e+03
----------------------------------------
NELL: -1.14e+04
W KL: 2.72e+03
S_in KL: 4.46e+01
B_in KL: 7.09e-01
S_out KL: 6.41e-03
B_out KL: 2.96e-02
Psi KL: 1.08e-01
----------------------------------------
LR: 0.01
Elapsed time (secs): 7.550106048583984
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 400 COMPLETE ===========
Obj: -8.73e+03
----------------------------------------
NELL: -1.16e+04
W KL: 2.81e+03
S_in KL: 4.33e+01
B_in KL: 4.00e-01
S_out KL: 7.27e-03
B_out KL: 2.42e-03
Psi KL: 9.08e-02
----------------------------------------
LR: 0.01
Elapsed time (secs): 10.068938970565796
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 500 COMPLETE ===========
Obj: -8.56e+03
----------------------------


=========== EPOCH 2100 COMPLETE ===========
Obj: -8.00e+03
----------------------------------------
NELL: -1.10e+04
W KL: 2.73e+03
S_in KL: 1.53e+02
B_in KL: 9.85e-01
S_out KL: 8.70e+01
B_out KL: 2.56e-01
Psi KL: 4.32e-02
----------------------------------------
LR: 0.01
Elapsed time (secs): 52.735055446624756
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 2200 COMPLETE ===========
Obj: -3.67e+03
----------------------------------------
NELL: -7.96e+03
W KL: 2.78e+03
S_in KL: 1.21e+03
B_in KL: 3.41e-01
S_out KL: 2.89e+02
B_out KL: 1.02e+01
Psi KL: 6.88e-02
----------------------------------------
LR: 0.01
Elapsed time (secs): 55.24310231208801
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 2300 COMPLETE ===========
Obj: -7.96e+03
-------------------------


=========== EPOCH 3900 COMPLETE ===========
Obj: -8.00e+03
----------------------------------------
NELL: -1.12e+04
W KL: 2.75e+03
S_in KL: 1.38e+02
B_in KL: 9.42e-02
S_out KL: 3.54e+02
B_out KL: 3.36e-01
Psi KL: 2.18e-01
----------------------------------------
LR: 0.01
Elapsed time (secs): 97.90993046760559
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 4000 COMPLETE ===========
Obj: -8.87e+03
----------------------------------------
NELL: -1.17e+04
W KL: 2.79e+03
S_in KL: 3.05e+00
B_in KL: 2.45e-01
S_out KL: 1.28e+01
B_out KL: 3.04e-01
Psi KL: 2.26e-01
----------------------------------------
LR: 0.01
Elapsed time (secs): 100.54162383079529
----------------------------------------
CPU cur memory used (GB): 6.91e+00
GPU_0 cur memory used (GB): 2.72e-01, max memory used (GB): 2.72e-01

=========== EPOCH 4100 COMPLETE ===========
Obj: -7.91e+03
-------------------------

## Plot some example data

In [18]:
# Helper formatting function
def format_box(ax):
    ax.set_aspect('equal', 'box')
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    plt.xlabel('property dim 1 (a.u.)')
    plt.ylabel('property dim 2 (a.u.)')

In [19]:
n_plot_vars = 1000 # Number of variables we plot the location of
ex_ind = 0

### Show the location of some example variables in property space

In [20]:
plt.figure(figsize=(7, 7))
ax = plt.subplot(1,1,1)
ax.plot(ind_props[ex_ind][0:n_plot_vars,0], ind_props[ex_ind][0:n_plot_vars,1], 'r.')
format_box(ax)

<IPython.core.display.Javascript object>

### Plot the mean and standard deviation of the true weight distribution

In [21]:
overlay_pts = True # True if we want to overlay neuron locations on plots

In [22]:
pts, dim_pts = list_grid_pts(grid_limits=np.asarray([[0, 1.0], [0, 1.0]]), n_pts_per_dim=[100,100])
pts = torch.tensor(pts, dtype=torch.float)

true_w_mn = true_priors.w_prior(pts).detach().cpu().numpy()
true_w_std = np.concatenate([d.std_f(pts).detach().cpu().numpy() for d in true_priors.w_prior.dists], axis=1)
true_w_mn = true_w_mn.reshape([100,100]).transpose()
true_w_std = true_w_std.reshape([100,100]).transpose()

plt.figure(figsize=(7, 7))
ax = plt.subplot(1,1,1)
im = ax.imshow(true_w_mn, origin='lower', extent=[0, 1.0, 0, 1.0])
mn_vmin, mn_vmax = im.get_clim() # Keep track of color limits
plt.colorbar(im)
format_box(ax)
plt.title('Mean')
if overlay_pts:
    ax.plot(ind_props[ex_ind][0:n_plot_vars,0], ind_props[ex_ind][0:n_plot_vars,1], 'r.')

plt.figure(figsize=(7, 7))
ax = plt.subplot(1,1,1)
im = ax.imshow(true_w_std, origin='lower', extent=[0, 1.0, 0, 1.0])
std_vmin, std_vmax = im.get_clim() # Keep track of color limits
plt.colorbar(im)
format_box(ax)
plt.title('Standard Deviation')
if overlay_pts:
    ax.plot(ind_props[ex_ind][0:n_plot_vars,0], ind_props[ex_ind][0:n_plot_vars,1], 'r.')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Plot the sampled weights for the example individual

In [23]:
plot_w_vls = ind_true_mdls[ex_ind].w.detach().numpy()[0:n_plot_vars]
plow_w_vls_scaled = (plot_w_vls - mn_vmin)/(mn_vmax - mn_vmin)
plot_w_clrs = cm.viridis(plow_w_vls_scaled)

plt.figure(figsize=(7, 7))
ax = plt.subplot(1,1,1)
ax.scatter(ind_props[ex_ind][0:n_plot_vars,0], ind_props[ex_ind][0:n_plot_vars,1], marker='.', color=plot_w_clrs)
format_box(ax)
plt.title('Sampled Weights')

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Sampled Weights')

## Plot the true low-d function and the true data from the example individual

In [24]:
n_plot_smps = 100

In [25]:
f_x_vls = torch.linspace(-2, 2, 100)
true_f_vls = ind_true_mdls[0].m(f_x_vls).detach().numpy()
f_x_vls = f_x_vls.detach().numpy()

ex_proj_x_vls = ind_true_mdls[ex_ind].project(x=ind_data[ex_ind][0]).detach().numpy()[0:n_plot_smps]
ex_y_vls = ind_data[ex_ind][1].detach().numpy()[0:n_plot_smps]

In [26]:
plt.figure(figsize=(9, 6.5))
plt.plot(ex_proj_x_vls, ex_y_vls, 'ro')
plt.plot(f_x_vls, true_f_vls, color='w')
plt.xlabel('Projected x (a.u.)')
plt.ylabel('y (a.u.)')

<IPython.core.display.Javascript object>

Text(0, 0.5, 'y (a.u.)')

### Plot the true low-d function and the predictions from the example individual

In [27]:
fit_type = 'comb'  # 'comb' or 'single'
exam_type = 'ip'

In [28]:
if fit_type == 'single':
    fit_rs = single_fit_rs
else:
    fit_rs = comb_fit_rs

if exam_type == 'sp':
    exam_priors = fit_rs['sp']['priors']
    exam_posts = [coll.posteriors for coll in fit_rs['sp']['vi_collections']]
    exam_vi_collections = fit_rs['sp']['vi_collections']
    exam_mdls = [coll.mdl for coll in fit_rs['sp']['vi_collections']]
    exam_logs = fit_rs['sp']['logs']
else:
    exam_priors = fit_rs['ip']['priors']
    exam_posts = [coll.posteriors for coll in fit_rs['ip']['vi_collections']]
    exam_vi_collections = fit_rs['ip']['vi_collections']
    exam_mdls = [coll.mdl for coll in fit_rs['ip']['vi_collections']]
    exam_logs = fit_rs['ip']['logs']

In [29]:
exam_w = exam_posts[ex_ind].w_post(ind_props[ex_ind])
exam_s_in = exam_posts[ex_ind].s_in_post(ind_props[ex_ind]).squeeze(axis=1)
exam_b_in = exam_posts[ex_ind].b_in_post(ind_props[ex_ind]).squeeze(axis=1)
exam_s_out = exam_posts[ex_ind].s_out_post(ind_props[ex_ind]).squeeze(axis=1)
exam_b_out = exam_posts[ex_ind].b_out_post(ind_props[ex_ind]).squeeze(axis=1)
exam_psi = exam_posts[ex_ind].psi_post.dists[0].mode(ind_props[ex_ind]).squeeze(axis=1)

In [30]:
n_exam_smps = 1000

In [31]:
x_exam = torch.tensor(sample_proj_data_from_interval(n_smps=n_exam_smps, 
                                                      w=ind_true_mdls[ex_ind].w.detach().numpy(), 
                                                      interval=[intervals[0], intervals[-1]]),
                       dtype=torch.float)

In [32]:
with torch.no_grad():
    
    # Determine which of our test points are in and out of the training distribution for this individual 
    x_true_proj = ind_true_mdls[ex_ind].project(x_exam).numpy()
    x_true_proj_for_int = ind_true_mdls[ex_ind].project(x_exam, apply_scales_and_biases=False).numpy().squeeze()
    
    x_within_train_dist = np.logical_and(x_true_proj_for_int >= intervals[ex_ind], 
                                         x_true_proj_for_int < intervals[ex_ind+1])
    x_outside_train_dist = np.logical_not(x_within_train_dist)
    
    # Get true mean and predicted mean for each data point for this individual 
    true_mns = ind_true_mdls[ex_ind].m(ind_true_mdls[ex_ind].project(x_exam)).numpy()
    pred_mns = exam_mdls[ex_ind].cond_mean(x=x_exam, w=exam_w, s_in=exam_s_in, b_in=exam_b_in, 
                                             s_out=exam_s_out, b_out=exam_b_out)
    x_pred_proj = exam_mdls[ex_ind].project(x=x_exam, w=exam_w, s_in=exam_s_in, b_in=exam_b_in)

In [33]:
align_vls = align_low_d_spaces(w_0=ind_true_mdls[ex_ind].w.detach().numpy(),
                               s_in_0=ind_true_mdls[ex_ind].s_in.detach().numpy(),
                               b_in_0=ind_true_mdls[ex_ind].b_in.detach().numpy(),
                               w_1=exam_w.detach().numpy(),
                               s_in_1=exam_s_in.detach().numpy(),
                               b_in_1=exam_b_in.detach().numpy(),
                               z_1=x_pred_proj.numpy())

x_pred_proj_aligned = align_vls[-1]

In [34]:
true_clrs = np.zeros([n_exam_smps,4])
true_clrs[:,-1] = 1.0
true_clrs[x_outside_train_dist, 0] = 1.0

pred_clrs = np.zeros([n_exam_smps,4])
pred_clrs[:, -1] = 1.0
pred_clrs[x_outside_train_dist, 0:3] = .5
#pred_clrs[x_outside_train_dist, -1] = 0 
pred_clrs[np.logical_not(x_outside_train_dist), 0] = .7


plt.figure(figsize=(9, 6.5))
a = plt.subplot(1,1,1)
a.scatter(x_true_proj, pred_mns, color=pred_clrs)
plt.ylim([-2.5, 2])

plt.plot(f_x_vls, true_f_vls, color='w')
plt.xlabel('Projected x (a.u.)')
plt.ylabel('y (a.u.)')


<IPython.core.display.Javascript object>

Text(0, 0.5, 'y (a.u.)')

### Plot the learned priors

In [35]:
pred_w_mn = exam_priors.w_prior(pts).detach().cpu().numpy()
pred_w_std = np.concatenate([d.std_f(pts).detach().cpu().numpy() for d in exam_priors.w_prior.dists], axis=1)

true_w_mn = true_priors.w_prior(pts).detach().cpu().numpy()

pred_mn_supp = np.concatenate([pred_w_mn, np.ones([pred_w_mn.shape[0], 1])], axis=1)
t = numpy.linalg.lstsq(pred_mn_supp, true_w_mn, rcond=None)
t = t[0]
pred_mn_al = np.matmul(pred_mn_supp, t)
pred_mn_al = pred_mn_al.reshape([100,100]).transpose()


t_std = t[0]
pred_w_std_al = np.zeros(pred_w_std.shape)
for i, std_i in enumerate(pred_w_std):
    pred_w_std_al[i, :] = np.abs(t_std)*std_i
pred_w_std_al = pred_w_std_al.reshape([100,100]).transpose()

In [36]:
pred_w_mn = pred_w_mn.reshape([100,100]).transpose()

In [37]:
plt.figure(figsize=(7, 7))
ax = plt.subplot(1,1,1)
im = ax.imshow(pred_mn_al, origin='lower', extent=[0, 1.0, 0, 1.0], vmin=mn_vmin, vmax=mn_vmax)
plt.colorbar(im)
format_box(ax)
plt.title('Estimated Mean')

plt.figure(figsize=(7, 7))
ax = plt.subplot(1,1,1)
im = ax.imshow(pred_w_std_al, origin='lower', extent=[0, 1.0, 0, 1.0], vmin=std_vmin, vmax=std_vmax)
plt.colorbar(im)
format_box(ax)
plt.title('Estimated Standard Deviation')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Text(0.5, 1.0, 'Estimated Standard Deviation')

## Look at fitting logs

In [38]:
for log in exam_logs:
    Fitter.plot_log(log)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Look at individual model parameters

In [39]:
exam_mdl = GNLRMdl(m = exam_vi_collections[ex_ind].mdl.m, w=exam_w, s_in=exam_s_in, b_in=exam_b_in, 
                  s_out=exam_s_out, b_out=exam_b_out, psi=exam_psi)

plt.figure()
GNLRMdl.compare_mdls(ind_true_mdls[ex_ind], exam_mdl)

<IPython.core.display.Javascript object>

[[-0.4566589]]


## Debug

In [40]:
exam_ind = 0

In [41]:
n_plot_vars = 10000

In [42]:
#plot_w_vls = w_true_proj[0:n_plot_vars]
plot_w_vls = exam_posts[exam_ind].w_post(ind_props[exam_ind]).detach().numpy()[0:n_plot_vars]
plow_w_vls_scaled = (plot_w_vls - np.min(plot_w_vls))/(np.max(plot_w_vls) - np.min(plot_w_vls))
plot_w_clrs = cm.viridis(plow_w_vls_scaled)

plt.figure(figsize=(7, 7))
ax = plt.subplot(1,1,1)
ax.scatter(locs[0:n_plot_vars,0], locs[0:n_plot_vars,1], marker='.', color=plot_w_clrs)
format_box(ax)
plt.title('Sampled Weights')

<IPython.core.display.Javascript object>

NameError: name 'locs' is not defined

In [ ]:
from probabilistic_model_synthesis.simulation import calc_mean_and_std_in_grid

In [ ]:
mn_i, std_i = calc_mean_and_std_in_grid(locs=locs, 
                                        vls=exam_posts[exam_ind].w_post(ind_props[exam_ind]).detach().numpy(),
                                        grid_dims=np.asarray([[0, 1.01, .05], [0, 1.01, .05]]) )

In [ ]:
plt.figure()
plt.imshow(mn_i.transpose(), origin='lower')

In [ ]:
exam_ind = 3

In [ ]:
locs = ind_props[exam_ind].numpy()

In [ ]:
std_vls = np.std(ind_data_new[exam_ind][0].numpy(), axis=0)

In [ ]:
mn_i, std_i = calc_mean_and_std_in_grid(locs=locs, 
                                        vls=std_vls,
                                        grid_dims=np.asarray([[0, 1.01, .05], [0, 1.01, .05]]) )

In [ ]:
plt.figure()
plt.imshow(mn_i)

In [ ]:
comb_fit_rs['sp']['vi_collections'][1].posteriors.b_in_post(ind_props[0])

In [ ]:
cand_smps = np.random.randn(10, 10)

In [ ]:
cand_smps.dtype

In [ ]:
cand_smps[5,:] = 1.0*cand_smps[5,:]

In [ ]:
cand_smps[5,:]

In [ ]:
cand_smps[5,:]